In [60]:
# Importing necessary libraries
import psycopg2
import matplotlib.pyplot as plt
import datetime
import matplotlib.dates as mdates
import pandas as pd
from IPython.display import display


# SQL query to retrieve data
def makeTable(forum_id, alpha, beta):
    conn = psycopg2.connect(
    #change these:
    dbname= "Viral Cascade",
    user="postgres",
    password="wapuh717",
    host="localhost",
    port="5432"
    )
    cursor = conn.cursor()

    
    query = f"select p.topic_id, count(distinct p.user_id) " \
            f"from posts p inner join topics t on t.topic_id = p.topic_id " \
             f"where forum_id = {forum_id} and classification2_topic >= 0.5 and length(content_post) > 10 group by p.topic_id"
    #classification_topic
    # and length(content_post) > 10
    # Executing the query
    cursor.execute(query)

    # Fetching all the rows
    threads = cursor.fetchall()


    betaID = []
    alphaID = []
    for x in threads:
        count = x[1]
        if count >= alpha*beta:
            betaID.append(x[0])
        elif count >= alpha:
            alphaID.append(x[0])

    print(f"Forum {forum_id}")
    print(f"Alpha {alpha}, beta {beta}")
    print(f"Forum {forum_id} has {len(betaID)} for alpha {alpha} and beta {beta}")

    startDates = []
    threadIDs = []
    endOfAlphas = []
    endOfBetas = []
    endOfLatests = []
    endOfAToB = []

    for x in betaID:
        id = x
        #Find the first post = start
        query = f"SELECT min(dateadded_post) FROM posts WHERE topic_id = {id}"
        cursor.execute(query)
        rows = cursor.fetchall()
        start = rows[0]
        start = start[0]
        startDates.append(start)

        #print(f"For thread number {id}")
        query = f"SELECT user_id, min(dateadded_post) date " \
                f"FROM (SELECT * FROM posts WHERE topic_id = {id} ORDER BY dateadded_post) " \
                f"GROUP BY user_id"
        # and length(content_post) > 10 
        # Executing the query
        cursor.execute(query)
        rows = cursor.fetchall()
        y = []
        for z in rows:
            y.append(z[1])
        ends = sorted(y)
        tdBeta = ends[beta*alpha-1] - start
        tdAlpha = ends[alpha] - start
        tdLatest = ends[-1] - start

        endOfBetas.append(tdBeta)
        endOfAlphas.append(tdAlpha)
        endOfLatests.append(tdLatest)

        #time delta values:
        tdSoloBeta = tdBeta - tdAlpha
        endOfAToB.append(tdSoloBeta)
        threadIDs.append(id)

        #print(f"Thread {id} has beta {tdBeta}")
        #print(tdAlpha)
        #print(tdSoloBeta)

    dict = {"Thread ID": threadIDs, "Start to Alpha DeltaT":endOfAlphas, "Alpha to Beta DeltaT":endOfAToB, "Start to Beta DeltaT":endOfBetas}
    df = pd.DataFrame(dict)
    display(df)
    return threadIDs, endOfAlphas, endOfAToB, endOfBetas
    cursor.close()
    conn.close()



In [61]:
from datetime import timedelta
deltaTs = [timedelta(days=30), timedelta(days=60), timedelta(days=90), timedelta(days=180)]

#2, 4, 8, 9

# Creating a cursor object
forum_ids = [2,4,8,9]
#All Forums: 1,2,3,4,5,6,7,8,9,10,11
#Has Results: 2,5,8,9,11
#No Results: 1,3,4,6,7,10
unique_user_requirement = 10
alphas = [10]
betas = [5]

for forum_id in forum_ids:
    for alpha in alphas:
        for beta in betas:
                threads, alphaDTs, betaDTs, alphabetaDTs = makeTable(forum_id, alpha, beta)

                a30 = []
                a60 = []
                a90 = []
                a180 = []


                b30 = []
                b60 = []
                b90 = []
                b180 = []

                ab30 = []
                ab60 = []
                ab90 = []
                ab180 = []

                for i in range(len(alphaDTs)):
                        a30.append(1 if alphaDTs[i]<=deltaTs[0] else 0)
                        a60.append(1 if alphaDTs[i]<=deltaTs[1] else 0)
                        a90.append(1 if alphaDTs[i]<=deltaTs[2] else 0)
                        a180.append(1 if alphaDTs[i]<=deltaTs[3] else 0)

                        b30.append(1 if betaDTs[i]<=deltaTs[0] else 0)
                        b60.append(1 if betaDTs[i]<=deltaTs[1] else 0)
                        b90.append(1 if betaDTs[i]<=deltaTs[2] else 0)
                        b180.append(1 if betaDTs[i]<=deltaTs[3] else 0)

                        ab30.append(1 if alphabetaDTs[i]<=deltaTs[0] else 0)
                        ab60.append(1 if alphabetaDTs[i]<=deltaTs[1] else 0)
                        ab90.append(1 if alphabetaDTs[i]<=deltaTs[2] else 0)
                        ab180.append(1 if alphabetaDTs[i]<=deltaTs[3] else 0)

    

                dict = {"Thread ID":threads, "30DT a":a30, "30DT b":b30, "30DT ab":ab30, "60DT a":a60, "60DT b":b60, "60DT ab":ab60, 
                        "90DT a":a90, "90DT b":b90, "90DT ab":ab90, "180DT a":a180, "180DT b":b180, "180DT ab":ab180}
                df = pd.DataFrame(dict)
                display(df)

                dict = {"30DT a":f"{sum(a30)/len(a30):.2%}", "30DT b":f"{sum(b30)/len(b30):.2%}", "30DT ab":f"{sum(ab30)/len(ab30):.2%}", 
                        "60DT a":f"{sum(a60)/len(a60):.2%}", "60DT b":f"{sum(b60)/len(b60):.2%}", "60DT ab":f"{sum(ab60)/len(ab60):.2%}", 
                        "90DT a":f"{sum(a90)/len(a90):.2%}", "90DT b":f"{sum(b90)/len(b90):.2%}", "90DT ab":f"{sum(ab90)/len(ab90):.2%}",
                        "180DT a":f"{sum(a180)/len(a180):.2%}", "180DT b":f"{sum(b180)/len(b180):.2%}", "180DT ab":f"{sum(ab180)/len(ab180):.2%}"}
                df = pd.DataFrame(dict, index=[0])

                print("This table displays the percentage of threads that reach start alpha/alpha to beta/start to beta " \
                      "(respectively a/b/ab) within a certain delta t amount of days.")
                display(df)
                #dict = {"30DT a":(sum(a30)/len(30)), "30DT b":b30, "30DT ab":ab30, "60DT a":a60, "60DT b":b60, "60DT ab":ab60, 
                #        "90DT a":a90, "90DT b":b90, "90DT ab":ab90, "180DT a":a180, "180DT b":b180, "180DT ab":ab180}
                #print(f"Average % of threads that reached 30 delta T for alpha is {}")




Forum 2 has 121 for alpha 10 and beta 5None


,Thread ID,Start to Alpha DeltaT,Alpha to Beta DeltaT,Start to Beta DeltaT
0,589,0 days 00:18:15,0 days 09:40:32,0 days 09:58:47
1,602,0 days 00:07:44,0 days 09:24:57,0 days 09:32:41
2,609,3 days 19:53:35,10 days 06:02:51,14 days 01:56:26
3,835,1 days 16:57:19,39 days 06:17:16,40 days 23:14:35
4,836,1 days 00:59:31,113 days 08:46:28,114 days 09:45:59
...,...,...,...,...
116,43804,0 days 01:58:19,2 days 02:18:18,2 days 04:16:37
117,43909,0 days 11:27:04,5 days 01:14:54,5 days 12:41:58
118,43943,0 days 00:08:35,2 days 00:42:11,2 days 00:50:46
119,43954,0 days 00:06:30,0 days 16:51:17,0 days 16:57:47


,Thread ID,30DT a,30DT b,30DT ab,60DT a,60DT b,60DT ab,90DT a,90DT b,90DT ab,180DT a,180DT b,180DT ab
0,589,1,1,1,1,1,1,1,1,1,1,1,1
1,602,1,1,1,1,1,1,1,1,1,1,1,1
2,609,1,1,1,1,1,1,1,1,1,1,1,1
3,835,1,0,0,1,1,1,1,1,1,1,1,1
4,836,1,0,0,1,0,0,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,43804,1,1,1,1,1,1,1,1,1,1,1,1
117,43909,1,1,1,1,1,1,1,1,1,1,1,1
118,43943,1,1,1,1,1,1,1,1,1,1,1,1
119,43954,1,1,1,1,1,1,1,1,1,1,1,1


This table displays the percentage of threads that reach start alpha/alpha to beta/start to beta (respectively a/b/ab) within a certain delta t amount of days.


,30DT a,30DT b,30DT ab,60DT a,60DT b,60DT ab,90DT a,90DT b,90DT ab,180DT a,180DT b,180DT ab
0,99.17%,95.04%,94.21%,100.00%,98.35%,98.35%,100.00%,98.35%,98.35%,100.00%,100.00%,100.00%


...

Forum 9 has 77 for alpha 10 and beta 5None


,Thread ID,Start to Alpha DeltaT,Alpha to Beta DeltaT,Start to Beta DeltaT
0,21542,216 days 00:00:00,0 days 00:00:00,216 days 00:00:00
1,21543,207 days 00:00:00,0 days 00:00:00,207 days 00:00:00
2,21544,645 days 00:00:00,0 days 00:00:00,645 days 00:00:00
3,21546,197 days 00:00:00,0 days 00:00:00,197 days 00:00:00
4,21547,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00
...,...,...,...,...
72,25549,10 days 08:25:09,61 days 07:56:32,71 days 16:21:41
73,25618,24 days 11:17:28,102 days 16:49:36,127 days 04:07:04
74,25700,26 days 00:52:08,111 days 08:34:35,137 days 09:26:43
75,28427,11 days 15:02:49,65 days 14:58:41,77 days 06:01:30


,Thread ID,30DT a,30DT b,30DT ab,60DT a,60DT b,60DT ab,90DT a,90DT b,90DT ab,180DT a,180DT b,180DT ab
0,21542,0,1,0,0,1,0,0,1,0,0,1,0
1,21543,0,1,0,0,1,0,0,1,0,0,1,0
2,21544,0,1,0,0,1,0,0,1,0,0,1,0
3,21546,0,1,0,0,1,0,0,1,0,0,1,0
4,21547,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,25549,1,0,0,1,0,0,1,1,1,1,1,1
73,25618,1,0,0,1,0,0,1,0,0,1,1,1
74,25700,1,0,0,1,0,0,1,0,0,1,1,1
75,28427,1,0,0,1,0,0,1,1,1,1,1,1


This table displays the percentage of threads that reach start alpha/alpha to beta/start to beta (respectively a/b/ab) within a certain delta t amount of days.


,30DT a,30DT b,30DT ab,60DT a,60DT b,60DT ab,90DT a,90DT b,90DT ab,180DT a,180DT b,180DT ab
0,70.13%,33.77%,7.79%,80.52%,41.56%,20.78%,84.42%,49.35%,33.77%,84.42%,90.91%,71.43%
